Google Earth Engine Python API code to convert a multiband image to an image collection.

This code specifically shows how to take precipitation data released by [India Meteorological Department](http://imdpune.gov.in/Clim_Pred_LRF_New/Grided_Data_Download.html) and convert it into a form that is usable in Earth Engine. 

Another notebook in this repository `imd_to_geotiff.ipynb` takes the gridded binary data and converts it to a GeoTiff file. Each geotiff file has 1 band per day for a given year. Once uploaded to Earth Engine, this notebook shows, how to take each band and write it to its separate asset, resulting in an ImageCollection object which is the preferred container to work with time series data such as this.

In [ ]:
import ee
ee.Authenticate()

In [5]:
ee.Initialize()

In [6]:
year = 2017
imageId = 'users/ujavalgandhi/external/imd/{}'.format(year)
outputCollectionId =  'users/ujavalgandhi/imd/rainfall'

## Create a Collection

In [7]:
image = ee.Image(imageId)
daysList = ee.List.sequence(0, image.bandNames().length().subtract(1))

def bandToDay(day):
    dayBand = image.select([day]).rename('rainfall')
    date = ee.Date.fromYMD(year, 1, 1).advance(day, 'day')
    return dayBand.set('system:time_start', date.millis())

dailyImages = daysList.map(bandToDay)

dailyCol = ee.ImageCollection.fromImages(dailyImages)

## Export All Images

Exports are done via the ``ee.batch`` module. 

In [8]:
image_ids = dailyCol.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Total images:  365


In [ ]:
for i, image_id in enumerate(image_ids):
    exportImage = ee.Image(dailyCol.filter(ee.Filter.eq('system:index', image_id)).first())
    exportAssetId = '{}/{}_{}'.format(outputCollectionId, year, i+1)
    task = ee.batch.Export.image.toAsset(**{
        'image': exportImage,
        'description': 'Image to Asset {}'.format(i+1),
        'assetId': exportAssetId,
        'scale': exportImage.projection().nominalScale().getInfo(),
        'region': exportImage.geometry().getInfo()['coordinates'],
        'maxPixels': 1e10
    })
    task.start()
print('Started Task: ', i+1)